# Create Spectroscopic Aperture Correction Reference Files

This notebook shows how to create spectroscopic aperture correction reference files for NIRCam WFSS starting from a table of aperture correction values.

In [ ]:
from astropy.io import ascii, fits
from astropy.table import Table
from jwst.datamodels.apcorr import NrcWfssApcorrModel
import matplotlib.pyplot as plt
import numpy as np

Input ascii table

In [ ]:
table_file = 'AperCor.table'

## Read in the input table

In [ ]:
table = ascii.read(table_file)

In [ ]:
table.columns

### Get a list of the extraction sizes in the table

In [ ]:
sizes_all = sorted(list(set(table['Width'])))

Input table Widths are in arcseconds. Change to pixels.

In [ ]:
pixel_scale = 0.06  # arcsec/pixel

In [ ]:
sizes_pix_all = (np.array(sizes_all) / pixel_scale).astype(np.int)

In the reference file, the 'size' column is specified to be an unsigned 8 bit integer. This means that the maximum value we can have in there is 255. The mean aperture correction (across wavelengths) for an extraction width of 255 pixels is about 1.011. This is small enough that it should be fine to ignore larger extraction sizes in the reference file. So cut down the table so that 255 pixels is the maximum size.

In [ ]:
last = sizes_pix_all <= 255.1
sizes_pix = sizes_pix_all[last]
sizes = np.array(sizes_all)[last]

In [ ]:
sizes_pix

In [ ]:
sizes

### Find the wavelength values used in the table

In [ ]:
wavelengths = sorted(list(set(table['Wavelength'])))

In [ ]:
print(len(wavelengths), min(wavelengths), max(wavelengths))

## Plot data as a sanity check

In [ ]:
# Put the 1-pixel extraction height on a separate plot since it's so much
# larger than those for the other apertures
f, a = plt.subplots(figsize=(6,7))
good = np.where(table['Width'] == sizes[0])
size_waves = table['Wavelength'][good]
size_apcor = table['AperCor'][good]
size_label = sizes[0] / pixel_scale
a.plot(size_waves, size_apcor, label='{} pix'.format(size_label))
a.set_ylabel('AperCor')
a.set_xlabel('Wavelength (micron)')
a.set_ylim(3, 15)
a.legend()

In [ ]:
# Plot curves for the next few extraction heights
f, a = plt.subplots(figsize=(12,14))
for size_val in sizes[1:10]:
    good = np.where(table['Width'] == size_val)
    size_waves = table['Wavelength'][good]
    size_apcor = table['AperCor'][good]
    size_label = size_val / pixel_scale
    a.plot(size_waves, size_apcor, label='{} pix'.format(size_label))
a.set_ylabel('AperCor')
a.set_xlabel('Wavelength (micron)')
a.set_ylim(1, 2.5)
a.legend()

In [ ]:
# Plot the curve for extraction height = 255 pixels. If this is close to 1, then it
# will be safe to ignore the curves for larger extraction heights.
f, a = plt.subplots()
goodsize = 15.3
good = np.isclose(table['Width'], goodsize, rtol=0, atol=0.001)
a.plot(table['Wavelength'][good], table['AperCor'][good])
a.set_ylabel('Aperture Correction')
a.set_xlabel('Wavelength (micron)')

### Rearrange Apcorr values into the expected 2D table

In [ ]:
nelem_size = len(sizes)
nelem_wl = len(wavelengths)

In [ ]:
apcor = np.zeros((nelem_size, nelem_wl))

In [ ]:
row = 0
for size_val in sizes:
    good = np.where(table['Width'] == size_val)
    size_waves = table['Wavelength'][good]
    size_apcor = table['AperCor'][good]
    apcor[row, :] = size_apcor
    row += 1

In [ ]:
# Make sure all elements have been populated. There should be no zeros left in the array
np.min(apcor)

In [ ]:
# Uncertainties - keep zero for now
apcor_err = np.zeros((nelem_size, nelem_wl))

## Filter and Pupil lists

In [ ]:
filters = ['F322W2', 'F277W', 'F356W', 'F444W', 'F250M', 'F300M',
           'F335M', 'F360M', 'F410M', 'F430M', 'F460M', 'F480M']
pupils = ['GRISMR', 'GRISMC']

## Create the full table to populate the model

Data table created following the example in the jwst package's [photom reference file creation page on JDox](https://jwst-pipeline.readthedocs.io/en/latest/jwst/photom/reference_files.html#constructing-a-photom-reference-file).

Each characteristic should be put into a numpy array or list with the appropriate data type

In [ ]:
num_rows = len(filters) * 2

mod_filter = filters * 2
mod_pupil = [pupils[0]] * len(filters) + ([pupils[1]] * len(filters))
mod_wavelength = np.array([wavelengths] * num_rows, dtype=np.float32)
mod_nelem_wl = np.array([nelem_wl] * num_rows, dtype=np.int16)
mod_size = np.array([list(sizes_pix)] * num_rows, dtype=np.uint8)
mod_nelem_size = np.array([nelem_size] * num_rows, dtype=np.int16)
mod_apcorr = np.array([apcor] * num_rows, dtype=np.float32)
mod_apcorr_err = np.array([apcor_err] * num_rows, dtype=np.float32)

Combine all fields into a single list, and then convert to an ndarray with the proper data types and sizes

In [ ]:
data_list = [(mod_filter[i], mod_pupil[i], mod_wavelength[i], mod_nelem_wl[i], mod_size[i],
              mod_nelem_size[i], mod_apcorr[i], mod_apcorr_err[i]) for i in range(num_rows)]

In [ ]:
data = np.array(data_list,
                dtype=[('filter', 'S12'),
                       ('pupil', 'S15'),
                       ('wavelength', '<f4', (nelem_wl,)),
                       ('nelem_wl', '<i2'),
                       ('size', 'u1', (nelem_size,)),
                       ('nelem_size', '<i2'),
                       ('apcorr', '<f4', (nelem_size, nelem_wl)),
                       ('apcorr_err', '<f4', (nelem_size, nelem_wl))])

## Create Model Instance

and populate with the ndarray from above

In [ ]:
model = NrcWfssApcorrModel(apcorr_table=data)

In [ ]:
# Specify units
model.apcorr_table.columns['wavelength'].unit = 'um'
model.apcorr_table.columns['size'].unit = 'pixels'

### Populate metadata

In [ ]:
model.meta.telescope = 'JWST'
model.meta.reftype = 'APCORR'
model.meta.pedigree = 'GROUND'
model.meta.description = ('This is the initial version of the spectroscopic aperture correction reffile '
                          'for NIRCam WFSS.')
model.meta.author = 'B. Hilbert'
model.meta.useafter = '2014-01-01T00:00:01'
model.meta.instrument.name = 'NIRCAM'
model.meta.instrument.detector = 'ANY'

In [ ]:
model.meta.exposure.type = 'NRC_WFSS'
model.meta.exposure.p_exptype = 'NRC_GRISM|NRC_WFSS|'

In [ ]:
model.history.append(('This spectroscopic aperture correction file for NIRCam WFSS was created using '
                      'encircled energy calculations on monochromatic model PSFs from WebbPSF. Aperture '
                      'correction values were calculated for extraction heights ranging between 1 and '
                      '255 pixels. Calculations were done by N. Pirzkal using WebbPSF 0.9.0. Packaging of '
                      'the aperture correction results into CRDS reference files done by B. Hilbert in '
                      'spec_apcorr_files_from_table.ipynb in the spacetelescope/nircam-calib repository.'))

In [ ]:
model.save('nrc_wfss_apercorr.fits')